In [0]:
%sql
DROP TABLE IF EXISTS demo;
CREATE OR REPLACE TABLE demo(id INT,valid BOOLEAN,name STRING);

In [0]:
%sql
INSERT INTO demo VALUES (1,true,"okk");

num_affected_rows,num_inserted_rows
1,1


check if schema enforcement really works

In [0]:
%sql
INSERT INTO demo VALUES (2,True,"yes","by")
--  schema mismatch error

com.databricks.sql.transaction.tahoe.DeltaAnalysisException: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: 9e7c4684-0604-4071-81b6-a509121acb89).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: integer (nullable = true)
-- valid: boolean (nullable = true)
-- name: string (nullable = true)


Data schema:
root
-- id: integer (nullable = true)
-- valid: boolean (nullable = true)
-- name: string (nullable = true)
-- col4: string (nullable = true)

         
	at com.databricks.sql.transaction.tahoe.MetadataMismatchErrorBuilder.finalizeAndThrow(DeltaErrors.scala:3772)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata(ImplicitMetadataOpe

In [0]:
%sql
INSERT INTO demo VALUES ("one",True,"yes")
--  data type error

org.apache.spark.SparkNumberFormatException: [CAST_INVALID_INPUT] The value 'one' of the type "STRING" cannot be cast to "INT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22018
== SQL (line 1, position 1) ==
INSERT INTO demo VALUES ("one",True,"yes")
^^^^^^^^^^^^^^^^

	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidInputInCastToNumberError(QueryExecutionErrors.scala:193)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.withException(UTF8StringUtils.scala:51)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.toIntExact(UTF8StringUtils.scala:34)
	at org.apache.spark.sql.catalyst.expressions.Cast.$anonfun$castToInt$2(Cast.scala:950)
	at org.apache.spark.sql.catalyst.expressions.Cast.$anonfun$castToInt$2$adapted(Cast.scala:950)
	at org.apache.spark.sql.catalyst.expressi

In [0]:
%sql
INSERT INTO demo VALUES ("1",True,"yes")
--  this will work automatically convert string 1 to int 1

num_affected_rows,num_inserted_rows
1,1


In [0]:
%python
data= [(1,True,"yupp","or nope")]
df=spark.createDataFrame(data,schema="id INT,valid Boolean, name String, suprise String")
df.display()

id,valid,name,suprise
1,true,yupp,or nope


In [0]:
%python

df.write\
        .option("mergeSchema","true")\
        .mode("append")\
        .saveAsTable("demo")

In [0]:
%sql
select * from demo

id,valid,name,suprise
1,true,yupp,or nope
1,true,okk,null
1,true,yes,null


###There are limits of schema evolution
- It allows you to expand the schema
- It doesnt allows you to change the existing schema

In [0]:
%python
data= [("one",True,"yupp","or nope")]
df=spark.createDataFrame(data,schema="id INT,valid Boolean, name String, suprise String")


df.write\
        .option("mergeSchema","true")\
        .mode("append")\
        .saveAsTable("demo")

---------------------------------------------------------------------------
PySparkTypeError                          Traceback (most recent call last)
File <command-2497773069427487>, line 2
      1 data= [("one",True,"yupp","or nope")]
----> 2 df=spark.createDataFrame(data,schema="id INT,valid Boolean, name String, suprise String")
      5 df.write\
      6         .option("mergeSchema","true")\
      7         .mode("append")\
      8         .saveAsTable("demo")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1605, in SparkSession.createDataFrame(self, data, schema, samplingRatio, verifySchema)
   1600 if has_

#Merge for whole spark session

In [0]:
%sql
SET spark.databricks.delta.schema.autoMerge.enabled=True

key,value
spark.databricks.delta.schema.autoMerge.enabled,True


In [0]:
%sql
INSERT INTO demo VALUES (2,True,"ok","bye","hello","yoo")

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from demo

id,valid,name,suprise,col5,col6
2,true,ok,bye,hello,yoo
1,true,yupp,or nope,null,null
1,true,okk,null,null,null
1,true,yes,null,null,null


#In some special case if we want to change the scheme then we can do OverWrite schema

In [0]:
%python
data= [("one","yupp","or nope")]
df=spark.createDataFrame(data,schema="id String, c1 String, c2 String")


df.write\
        .option("overWriteSchema","true")\
        .mode("overwrite")\
        .saveAsTable("demo")

In [0]:
%sql
select * from demo

id,c1,c2
one,yupp,or nope
